# Query Google Earth Engine (GEE) for reference DEM for the `skysat_triplet_pipeline`

In [1]:
import os
import glob
import ee
import sys

## Define input and output files

In [ ]:
buffer = 2e3 # [m] how much to buffer the AOI before clipping DEM 

site_name = 'Banner'
site_dir = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/{site_name}'
out_dir = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/{site_name}/refdem/'
code_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat_stereo_snow'

# Import utility functions
sys.path.append(os.path.join(code_dir, 'skysat_stereo_snow'))
import query_refdem_utils as q


## Calculate image bounds and buffer

In [ ]:
dates = [x for x in sorted(glob.glob(os.path.join(site_dir, '20*'))) 
         if os.path.exists(os.path.join(x, 'SkySatScene'))]

meta_fns = []
for date in dates:
    # Load metadata file names
    fns = sorted(glob.glob(os.path.join(site_dir, date, 'SkySatScene', '*_metadata.json')))
    meta_fns += fns

# Create clipping region
bbox, epsg_utm = q.create_bbox_from_meta(meta_fns, buffer)


## Initialize GEE

In [4]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Intialize()

## Query GEE for DEM, clip, save to file

In [ ]:
# Make sure output directory exists
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    print('Made directory for output file:', out_dir)
# Define output file name
dem_fn = os.path.join(out_dir, f"{site_name}_COPDEM_buffer{int(buffer/1e3)}km.tif")
# Query and download DEM
dem_out_fn = q.query_gee_for_copdem(bbox, out_fn=dem_fn, crs=epsg_utm)

## Optional: coregister and merge with high-resolution DEM used for coregistration

In [ ]:
if site_name=='JacksonPeak':
    coregdem_fn = os.path.join(site_dir, 'refdem', 'USGS_LPC_ID_FEMAHQ_2018_D18_merged_filtered_UTM11.tif')
else:
    coregdem_fn = os.path.join(site_dir, 'refdem', f"{site_name}_REFDEM_WGS84.tif")

# Fill holes in high-res DEM
coregdem_fill_fn = q.fill_nodata(coregdem_fn)

# Coregister COPDEM to the high-res DEM
copdem_fill_coreg_fn = q.coregister_dems(coregdem_fill_fn, dem_out_fn)
    
# Merge DEMs: Overlay the COPDEM with the lidar DEM
merged_dem_fn = os.path.join(os.path.dirname(dem_out_fn), f'{site_name}_refdem_lidar_COPDEM_merged.tif')
_ = q.merge_dems(coregdem_fill_fn, copdem_fill_coreg_fn, out_fn=merged_dem_fn)